In [3]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# -------------------- Parameters --------------------
CL = 0.99
z = 2.3263478740408408
total_value = 1_000_000.0
weights = np.array([0.5, 0.3, 0.2])  #  weights of asset A , B, C

# 10-day liquidity add-on: "constant" or "sqrt10"
ADDON_SCALING_FOR_10D = "constant"   # change to "sqrt10" if you prefer √10 scaling

# -------------------- Load returns --------------------
# loading return file
try:
    returns = pd.read_csv("sample_data/3assets_returns.csv", index_col=0, parse_dates=True)
    assets = list(returns.columns)
    if len(assets) != len(weights):
        raise ValueError("Number of assets in returns data does not match the number of weights.")
except Exception as e:
    print(f"Error loading returns data: {e}")
    # Fallback or alternative loading/processing if needed
    # For now, we'll re-raise the error as it's fundamental to the calculation
    raise


# -------------------- Base VaR (Normal / parametric) --------------------
cov = returns.cov().values
sigma_p = float(np.sqrt(weights @ cov @ weights.T))
base_var_1d_normal = z * sigma_p * total_value
base_var_10d_normal = base_var_1d_normal * np.sqrt(10)  # √time scaling

# -------------------- Liquidity parameters --------------------
# (editing these with your actual HalfSpread/Impact/TTL values)
values = total_value * weights
liq_params = pd.DataFrame({
    "Asset": assets,
    "Value": values,
    "HalfSpread_bps": [10.0, 15.0, 8.0],  # example half-spreads in bps
    "TTL_Days": [3, 3, 3],                # example time-to-liquidate (days)
    "Impact_bps": [5.0, 8.0, 4.0],        # example impact in bps
})

# -------------------- Liquidity add-ons (per asset & total) --------------------
# Add-on = (bps/1e4) * Value * √TTL
liq_params["AddOn_spread_sqrtTTL"] = (
    (liq_params["HalfSpread_bps"] / 1e4) * liq_params["Value"] * np.sqrt(liq_params["TTL_Days"])
)
liq_params["AddOn_impact"] = (
    (liq_params["Impact_bps"] / 1e4) * liq_params["Value"] * np.sqrt(liq_params["TTL_Days"])
)
liquidity_addon_total_1d = float(liq_params["AddOn_spread_sqrtTTL"].sum() + liq_params["AddOn_impact"].sum())

# Option for 10-day liquidity add-on
if ADDON_SCALING_FOR_10D.lower() == "sqrt10":
    liquidity_addon_total_10d = liquidity_addon_total_1d * np.sqrt(10)
else:
    liquidity_addon_total_10d = liquidity_addon_total_1d  # "constant"

# -------------------- Liquidity-adjusted VaR (L-VaR) --------------------
lvar_1d_total  = base_var_1d_normal  + liquidity_addon_total_1d
lvar_10d_total = base_var_10d_normal + liquidity_addon_total_10d

# -------------------- Output tables --------------------
asset_table = liq_params.set_index("Asset")

metrics_table = pd.DataFrame({
    "Metric": [
        "Base_VaR_1d_Normal",
        "Base_VaR_10d_Normal",
        "Liquidity_AddOn_1d",
        "Liquidity_AddOn_10d",
        "LVaR_1d_Total",
        "LVaR_10d_Total",
    ],
    "Value": [
        base_var_1d_normal,
        base_var_10d_normal,
        liquidity_addon_total_1d,
        liquidity_addon_total_10d,
        lvar_1d_total,
        lvar_10d_total,
    ],
}).set_index("Metric")

# -------------------- Save / Display --------------------
asset_csv = "sample_data/liquidity_addons_assets.csv"
metrics_csv = "sample_data/liquidity_addons_metrics.csv"
asset_table.round(2).to_csv(asset_csv)
metrics_table.round(2).to_csv(metrics_csv)

print("\n=== Liquidity Add-ons (per asset) ===")
print(asset_table.round(2))
print("\n=== Liquidity Metrics ===")
print(metrics_table.round(2))

print(f"\nSaved: {asset_csv}")
print(f"Saved: {metrics_csv}")


=== Liquidity Add-ons (per asset) ===
                   Value  HalfSpread_bps  TTL_Days  Impact_bps  \
Asset                                                            
Asset_A_Return  500000.0            10.0         3         5.0   
Asset_B_Return  300000.0            15.0         3         8.0   
Asset_C_Return  200000.0             8.0         3         4.0   

                AddOn_spread_sqrtTTL  AddOn_impact  
Asset                                               
Asset_A_Return                866.03        433.01  
Asset_B_Return                779.42        415.69  
Asset_C_Return                277.13        138.56  

=== Liquidity Metrics ===
                        Value
Metric                       
Base_VaR_1d_Normal   20187.01
Base_VaR_10d_Normal  63836.93
Liquidity_AddOn_1d    2909.85
Liquidity_AddOn_10d   2909.85
LVaR_1d_Total        23096.86
LVaR_10d_Total       66746.78

Saved: sample_data/liquidity_addons_assets.csv
Saved: sample_data/liquidity_addons_metrics.csv
